In [1]:
from utils import *
import pymongo
import pandas as pd
import json

In [2]:
config = load_config()
mongo_client = pymongo.MongoClient(get_mongo_url(config))
db = mongo_client['ChameleonSimulator']

In [3]:
db.collection_names()

ipykernel_launcher:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.


['resource_pool']

In [4]:
rsrc_pool = db['resource_pool']

In [5]:
result = rsrc_pool.aggregate([
    {"$match": {"pool": "osg"}},
    {"$project": {
        "Machine": "$HOST_NAME (PHYSICAL)",
        "free_cpus": {"$subtract": ["$cpus", "$inuse_cpus"]}, 
        "free_memory": {"$subtract": ["$memory", "$inuse_memory"]}
    }},
    {"$match": { 
        "$and": [
            {"free_cpus": {"$gte": 1}},
            {"free_memory": {"$gte": 1}}
        ]}
    },
    {"$limit": 1}
])

result = pd.DataFrame(list(result))
print(result)
result.apply(lambda row: str(row['_id']), axis=1)
result = result.to_dict(orient='records')
# json.dumps(result)
# result['_id'] = [str(x) for x in result['_id'].to_list()]
# result = result.to_dict(orient="records")
# (result)
# result = rsrc_pool.update_one(
#     {'_id': result[0]['_id']}, 
#     {"$inc": {
#         "inuse_cpus": 8, 
#         "inuse_memory": 8
#         },
#         "$pull": {"backfill": {"GlobalJobId": "abc"}}
#     }
# )
# print(result)

                        _id                           Machine  free_cpus  \
0  60f8c4109ae3d7ea22b931b5  fa9dc77d7da1a9517f2a7fc1b5141824          1   

   free_memory  
0       122880  


In [6]:
result = rsrc_pool.aggregate([
    {"$match": {"$and": [ {"node_type": 'compute_haswell'}, {"pool": "chameleon"}, {"status": "inuse"} ] }},
    {"$limit": 10}
])
result = list(result)
print(result)

[{'_id': ObjectId('60f8c4109ae3d7ea22b931b6'), 'EVENT_TIME': '2018-03-14 11:40:05', 'EVENT_TIME_SEC': 86528405.0, 'HOST_NAME (PHYSICAL)': '3a9800c4b261fdc8595ed40da7568ccc', 'EVENT': 'UPDATE', 'PROPERTIES': "{'node_type': 'compute_haswell', 'placement.rack': 1, 'main_memory.ram_size': '134956859392', 'architecture.smt_size': '48', 'storage_devices.0.size': '250059350016', 'architecture.smp_size': '2', 'architecture.platform_type': 'x86_64'}", 'node_type': 'compute_haswell', 'status': 'inuse', 'pool': 'chameleon', 'cpus': 2, 'inuse_cpus': 0, 'memory': 131072, 'inuse_memory': 0, 'backfill': []}, {'_id': ObjectId('60f8c4109ae3d7ea22b931b7'), 'EVENT_TIME': '2018-03-14 11:40:05', 'EVENT_TIME_SEC': 86528405.0, 'HOST_NAME (PHYSICAL)': 'ef87ce346581cc2449c4c827bd3844b9', 'EVENT': 'UPDATE', 'PROPERTIES': "{'placement.rack': 1, 'node_type': 'compute_haswell', 'main_memory.ram_size': '134956859392', 'architecture.smp_size': '2', 'storage_devices.0.size': '250059350016', 'architecture.smt_size': '